# Notebook 2: Least Squares Regression and Scaling

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import glob
import pandas as pd
import sklearn
from src import ana_utils as utils

#np.set_printoptions(suppress=True)
from sklearn.linear_model import LinearRegression

np.random.seed(5)

Import datasets that were preprocessed in Notebook 1

In [2]:
wb_data = pd.read_csv("data/wb_data.csv", index_col="Country Name")
wb_data_short = pd.read_csv("data/wb_data_short.csv", index_col="Country Name")
whr_data = pd.read_csv("data/whr_data.csv", index_col="Country name")

# test: are the same countries present in each dataset?
print(sorted(list(wb_data.index))==sorted(list(whr_data.index)))

True


Split data into train and test set. We choose a 80/20 split, i.e. 120 countries in the training set and 30 countries in the test set.

In [3]:
test_size = 30
train, test, train_gt, test_gt = utils.split_data(wb_data_short, whr_data, test_size)

# verify set shapes
print(train.shape, test.shape, train_gt.shape, test_gt.shape)

#verify that data prder and groundtruth order and indices match 
print(list(train.index)==list(train_gt.index), list(test.index)==list(test_gt.index))


(120, 120) (30, 120) (120, 1) (30, 1)
True True


## Linear regression


Let's see how linear least squares regression performs on wb_data and wb_data_short (redundant indicators removed). We choose 2000-fold validation after noticing quite some variance for lower n.

In [4]:
least_squares = sklearn.linear_model.LinearRegression()

# full set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data, gt=whr_data, test_size=test_size, scaling="no_scaling", calc_adj_r_squared=True)
print("Mean loss (full set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients ((full set of indicators)):", avg_coefs[:10], "\n")

# manually reduced set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data_short, gt=whr_data, test_size=test_size, scaling="no_scaling", calc_adj_r_squared=True)
print("Mean test loss (reduced set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients (reduced set of indicators):", avg_coefs[:10])

Mean loss (full set of indicators): 15.306620501590588
Mean train loss (reduced set of indicators): 0.00013484961054526282
Adjusted R^2: 1.0003512885370727
The average size of the first ten coefficients ((full set of indicators)): [ 0.0153 -0.0078 -0.     -0.     -0.003   0.0638 -0.0094  0.0732  0.0743
 -0.087 ] 

Mean test loss (reduced set of indicators): 5.141473093039218
Mean train loss (reduced set of indicators): 0.044210642634051014
Adjusted R^2: 5.259811140144357
The average size of the first ten coefficients (reduced set of indicators): [ 0.0198 -0.0069  0.     -0.     -0.0054  0.4498  0.1879  0.0765 -0.021
  0.0121]


While linear regression performs better after manually removing redundancies, both of the results are still quite poor. 
We suspect multicolinearity to be a main reason for bad performance. 

But before starting to deal with multicolinearity, we want to normalize/standardize the data. This is because, in the end, we aim to compare coefficients. But as of now, coefficient sizes vary drastically, thus requiring rescaling of the features.

Hence, performing analysis also on the normalized/standardized data along the way is necessary to prevent us from developing a model that works only on non-normalized/non-standardized data. 

### Normalization 

Here, we normalize each row, using the L2 norm. That is, for each country the indicator values are scaled such that the sum of all squared indicator values is one.

In [5]:
least_squares = sklearn.linear_model.LinearRegression()

# full set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data, gt=whr_data, test_size=test_size, scaling="normalize", calc_adj_r_squared=True)
print("Mean loss (full set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients ((full set of indicators)):", avg_coefs[:10], "\n")

# manually reduced set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data_short, gt=whr_data, test_size=test_size, scaling="normalize", calc_adj_r_squared=True)
print("Mean test loss (reduced set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients (reduced set of indicators):", avg_coefs[:10])

Mean loss (full set of indicators): 29.734519256246376
Mean train loss (reduced set of indicators): 9.670897255292179e-27
Adjusted R^2: 1.0
The average size of the first ten coefficients ((full set of indicators)): [ 2.87389e+01 -1.36412e+01 -3.59000e-02 -1.63900e-01 -5.70000e-01
  6.21691e+01  2.74549e+01  6.69896e+01  5.53360e+00 -1.68528e+01] 

Mean test loss (reduced set of indicators): 5.102828042215271
Mean train loss (reduced set of indicators): 0.014454184245967004
Adjusted R^2: 2.3860196366686917
The average size of the first ten coefficients (reduced set of indicators): [ 19.8859  -5.841    0.9651  -0.3379  -3.751  109.2427 118.7904  24.4655
  -4.8925   1.2468]


### Standardization
Each value x is scaled with the formula $\frac{x-\mu}{\sigma}$

In [6]:
least_squares = sklearn.linear_model.LinearRegression()

# full set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data, gt=whr_data, test_size=test_size, scaling="standardize", calc_adj_r_squared=True)
print("Mean loss (full set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients ((full set of indicators)):", avg_coefs[:10], "\n")

# manually reduced set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data_short, gt=whr_data, test_size=test_size, scaling="standardize", calc_adj_r_squared=True)
print("Mean test loss (reduced set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients (reduced set of indicators):", avg_coefs[:10])

Mean loss (full set of indicators): 26.097408656554713
Mean train loss (reduced set of indicators): 2.732927590123229e-27
Adjusted R^2: 1.0
The average size of the first ten coefficients ((full set of indicators)): [ 0.6783 -0.2046  0.013  -0.0664 -0.057   2.2972  0.8979  1.2529 -0.1647
 -1.2653] 

Mean test loss (reduced set of indicators): 5.720586924572409
Mean train loss (reduced set of indicators): 0.09138588436231791
Adjusted R^2: 9.823388911945019
The average size of the first ten coefficients (reduced set of indicators): [ 0.55   -0.1159  0.0758 -0.0102 -0.178   5.0187  4.1688  0.8284 -0.3168
  0.0826]


### Conclusion for Standardization/Normalization
Both methods seem to worsen mean loss for the regression on the full dataset. However, they don't change the mean loss notably for the dataset containing the manually reduced set of indicators.  
Therefore, we will from now on normalize the data in order to make the resulting regression coefficients more interpretable.